In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -rf rl_experiments/


In [ ]:
!GIT_LFS_SKIP_SMUDGE=1 git clone https://github.com/isayevlab/rl_experiments.git


Cloning into 'rl_experiments'...
remote: Enumerating objects: 95, done.
remote: Total 95 (delta 0), reused 0 (delta 0), pack-reused 95 (from 1)
Receiving objects: 100% (95/95), 445.13 MiB | 40.54 MiB/s, done.
Resolving deltas: 100% (32/32), done.
Updating files: 100% (58/58), done.


In [ ]:
!pip install rdkit-pypi torch torchvision pandas matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 72.1 MB/s eta 0:00:00


In [ ]:
!pip install joblib


In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available


True


In [ ]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available

True


In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
import sys
sys.path.append('/content/rl_experiments/release')


In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
sys.path.append('../release')

In [ ]:
import numpy as np
import pandas as pd
from tqdm import trange
import torch
use_cuda = torch.cuda.is_available()

In [ ]:
import os
print(os.getcwd())


/content


In [ ]:
from data import GeneratorData, PredictorData
from stackRNN import StackAugmentedRNN
from utils import get_fp
from reinforcement import Reinforcement

In [ ]:
import joblib
from sklearn.ensemble import RandomForestClassifier as RFC
from predictor import VanillaQSAR

# Training the predictor

In [ ]:
from rdkit import RDLogger

# Disable RDKit warnings
RDLogger.DisableLog('rdApp.*')

np.random.seed(42)

pred_data = PredictorData('rl_experiments/data/egfr_with_pubchem.csv', get_features=get_fp)
model_instance = RFC
model_params = {'n_estimators': 250,
                'n_jobs': 10}
my_predictor = VanillaQSAR(model_instance=model_instance,
                           model_params=model_params,
                           ensemble_size=10)

In [ ]:
# Train the model and retrieve metrics
results, metrics_type = my_predictor.fit_model(pred_data, cv_split='random')



In [ ]:
# Print out the results and calculate the average F1 score
print(f"Metrics Type: {metrics_type}")
for i, metric in enumerate(results):
    print(f"Fold {i+1}: {metrics_type} = {metric:.4f}")

# Calculate and print the average F1 score
average_f1 = np.mean(results)
print(f"Average {metrics_type}: {average_f1:.4f}")


Metrics Type: F1 score
Fold 1: F1 score = 0.8571
Fold 2: F1 score = 0.8301
Fold 3: F1 score = 0.8549
Fold 4: F1 score = 0.8525
Fold 5: F1 score = 0.8350
Fold 6: F1 score = 0.8465
Fold 7: F1 score = 0.8343
Fold 8: F1 score = 0.8540
Fold 9: F1 score = 0.8505
Fold 10: F1 score = 0.8357
Average F1 score: 0.8451


In [ ]:
# uncomment to train predictor model...
my_predictor.fit_model(pred_data, cv_split='random')
my_predictor.save_model('/content/drive/MyDrive/egfr_rfc')

In [ ]:
# ...or use pretrained model
my_predictor.load_model('/content/drive/MyDrive/egfr_rfc')

# Dynamic threshold Original (steps: 0.005 and max= 1)

In [ ]:
import os
import sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from rdkit import Chem, DataStructs
from sklearn.ensemble import RandomForestClassifier as RFC
from utils import canonical_smiles, get_fp
from stackRNN import StackAugmentedRNN
from data import GeneratorData
from predictor import VanillaQSAR
from reinforcement import Reinforcement
import time
import contextlib
from rdkit import RDLogger

# Suppress RDKit warnings and errors
RDLogger.DisableLog('rdApp.*')

# A function to suppress stderr (error messages)
@contextlib.contextmanager
def suppress_stderr():
    # Redirect stderr to null
    with open(os.devnull, 'w') as devnull:
        old_stderr = sys.stderr
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stderr = old_stderr

# Modify the function where the SMILES processing happens
def estimate_and_update(generator, predictor, n_to_generate, threshold=None, batch_size=16,
                        plot_counts=False, plot=False, return_metrics=False, **kwargs):
    generated = []
    pbar = tqdm(range(n_to_generate))
    for i in pbar:
        pbar.set_description("Generating molecules...")
        with suppress_stderr():  # Suppress error messages during SMILES generation
            generated += generator.evaluate(gen_data, predict_len=120, batch_size=batch_size)

    with suppress_stderr():  # Suppress error messages during canonicalization
        sanitized = canonical_smiles(generated, sanitize=False, throw_warning=False)[:-1]
    unique_smiles, counts = np.unique(sanitized, return_counts=True)
    unique_smiles, counts = list(unique_smiles)[1:], counts[1:]

    smiles, prediction, nan_smiles = predictor.predict(unique_smiles, get_features=get_fp)

    if plot_counts:
        if plot:
            plt.hist(counts)
            plt.gca().set_yscale('log')
            plt.title('Distribution of counts of generated smiles')
            plt.xlabel('Counts observed')
            plt.show()
        max_counts = max(counts)
        if max_counts > 1:
            print('Trajectories with max counts:')
            for i in np.where(counts == max_counts)[0]:
                print('%d\t%s' % (counts[i], unique_smiles[i]))

    with suppress_stderr():  # Suppress error messages during prediction
        plot_hist(prediction, len(generated), threshold, plot=plot)

    valid_fraction = len(prediction) / len(generated)
    active_fraction = np.mean(prediction >= threshold)
    metrics = {'valid_fraction': valid_fraction,
               'active_fraction': active_fraction}

    if plot_counts:
        metrics['max_counts'] = max_counts
    if return_metrics:
        return smiles, prediction, metrics
    else:
        return smiles, prediction


def plot_hist(prediction, n_to_generate, threshold=None, plot=True):
    print("Mean value of predictions:", prediction.mean())
    print("Proportion of valid SMILES:", len(prediction)/n_to_generate)
    if plot:
        plt.hist(prediction, bins=20, edgecolor='black')
        if threshold is not None:
            plt.axvline(x=threshold, color="red")
        plt.xlabel('Predicted pIC50')
        plt.title('Distribution of predicted pIC50 for generated molecules')
        plt.show()

def update_threshold(cur_threshold, prediction, proportion=0.15, step=0.05):
    if (prediction >= cur_threshold).mean() >= proportion:
        new_threshold = min(cur_threshold + step, 1.0)
        print(f"Threshold increased to: {new_threshold:.2f}")
        return new_threshold
    else:
        return cur_threshold

def simple_moving_average(prev_values, new_value, ma_window_size=10):
    ma_value = sum(prev_values[-(ma_window_size-1):]) + new_value
    ma_value = ma_value / (len(prev_values[-(ma_window_size-1):]) + 1.)
    return ma_value

def main(n_iterations=400, n_policy=10, n_policy_replay=15, batch_size=16, n_fine_tune=None,
         seed=None, replay_data_path='/content/rl_experiments/data/gen_actives.smi',
         primed_path='/content/rl_experiments/checkpoints/generator/checkpoint_batch_training',
         save_path=None):

    if n_fine_tune is None:
        n_fine_tune = n_iterations

    if seed is not None:
        np.random.seed(seed)
        torch.manual_seed(seed)

    gen_data_path = '/content/rl_experiments/data/chembl_22_clean_1576904_sorted_std_final.smi'
    tokens = [' ', '<', '>', '#', '%', ')', '(', '+', '-', '/', '.', '1', '0', '3', '2', '5', '4',
              '7', '6', '9', '8', '=', 'a', '@', 'C', 'B', 'F', 'I', 'H', 'O', 'N', 'P', 'S', '[', ']',
              '\\', 'c', 'e', 'i', 'l', 'o', 'n', 'p', 's', 'r']
    global gen_data
    gen_data = GeneratorData(gen_data_path, delimiter='\t',
                             cols_to_read=[0], keep_header=True, tokens=tokens)

    hidden_size = 1500
    stack_width = 1500
    stack_depth = 200
    layer_type = 'GRU'
    optimizer = torch.optim.SGD
    lr = 0.0002
    generator = StackAugmentedRNN(input_size=gen_data.n_characters, hidden_size=hidden_size,
                                  output_size=gen_data.n_characters, layer_type=layer_type,
                                  n_layers=1, is_bidirectional=False, has_stack=True,
                                  stack_width=stack_width, stack_depth=stack_depth,
                                  use_cuda=torch.cuda.is_available(), optimizer_instance=optimizer, lr=lr)
    # Load the model with appropriate device mapping
    weights = torch.load(primed_path, map_location=lambda storage, loc: storage.cuda() if torch.cuda.is_available() else storage.cpu())
    generator.load_state_dict(weights)


    model_instance = RFC
    model_params = {'n_estimators': 250, 'n_jobs': 10}
    predictor = VanillaQSAR(model_instance=model_instance,
                            model_params=model_params,
                            model_type='classifier')
    predictor.load_model('/content/drive/MyDrive/egfr_rfc')

    def get_reward(smiles, predictor, threshold, invalid_reward=1.0, get_features=get_fp):
        mol, prop, nan_smiles = predictor.predict([smiles], get_features=get_features)
        if len(nan_smiles) == 1:
            return invalid_reward
        if prop[0] >= threshold:
            return 10.0
        else:
            return invalid_reward

    RL_model = Reinforcement(generator, predictor, get_reward)

    rl_losses = []
    rewards = []
    valid_fractions = []
    active_fractions = []
    thresholds = []
    n_to_generate = 50
    threshold = 0.05
    start = time.time()

    for i in range(n_iterations):
        print(f"{i+1} Training on replay instances...")
        thresholds.append(threshold)

        # Policy Gradient
        for j in trange(n_policy, desc=f" {i+1} Policy gradient..."):
            cur_reward, cur_loss = RL_model.policy_gradient(gen_data, threshold=threshold)
            rewards.append(simple_moving_average(rewards, cur_reward))
            rl_losses.append(simple_moving_average(rl_losses, cur_loss))

        # Estimate and Update
        smiles_cur, prediction_cur, metrics = estimate_and_update(RL_model.generator,
                                                                  RL_model.predictor,
                                                                  n_to_generate,
                                                                  batch_size=batch_size,
                                                                  threshold=0.75,
                                                                  plot_counts=False,
                                                                  return_metrics=True)
        valid_fractions.append(metrics['valid_fraction'])
        active_fractions.append(metrics['active_fraction'])

        # Update threshold
        threshold = update_threshold(threshold, prediction_cur)

    duration = time.time() - start
    print(f"Training duration: {duration} seconds")

    # Plotting the results
    plt.figure(figsize=(12, 6))
    plt.plot(rewards, label='Rewards')
    plt.title('Rewards Over Iterations')
    plt.xlabel('Iterations')
    plt.ylabel('Reward')
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(valid_fractions, label='Valid Fractions')
    plt.plot(active_fractions, label='Active Fractions')
    plt.plot(thresholds, label='Thresholds')
    plt.title('Fractions and Threshold Over Iterations')
    plt.xlabel('Iterations')
    plt.ylabel('Fraction / Threshold')
    plt.legend()
    plt.show()

if __name__ == '__main__':
    main(n_iterations=400)

<ipython-input-13-8492074f9397>:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(primed_path, map_location=lambda storage, loc: storage.cuda() if torc

1 Training on replay instances...


Generating molecules...: 100%|██████████| 50/50 [00:08<00:00,  5.85it/s]


Mean value of predictions: 0.0
Proportion of valid SMILES: 0.7826633165829145
2 Training on replay instances...


Generating molecules...: 100%|██████████| 50/50 [00:08<00:00,  6.04it/s]


...

Generating molecules...: 100%|██████████| 50/50 [00:08<00:00,  5.75it/s]


Mean value of predictions: 0.768421
Proportion of valid SMILES: 0.02375
Threshold increased to: 1.00
298 Training on replay instances...


Generating molecules...: 100%|██████████| 50/50 [00:08<00:00,  5.78it/s]


Mean value of predictions: 0.8
Proportion of valid SMILES: 0.015
Threshold increased to: 1.00
299 Training on replay instances...


 299 Policy gradient...:  90%|█████████ | 9/10 [00:37<00:04,  4.20s/it]